In [151]:
import numpy as np
import pandas as pd
columnNames = ["pbx_id", "calls", "minutes", "type", "year", "month", "day"]
dfSep = pd.read_csv("septUsage.csv", names = columnNames)
dfAgo = pd.read_csv("agoUsage.csv", names = columnNames)
dfJul = pd.read_csv("julUsage.csv", names = columnNames)
    

In [155]:
dfSep['type'].value_counts()

outgoing                  181172
incoming                   50339
forward                     4347
originate                   2760
voicemailexit               2079
queue-exit                    68
cisco-3way                    31
default                       18
conferencing-originate        13
confirm-callee                 1
Name: type, dtype: int64

In [156]:
dfAgo['type'].value_counts()

outgoing                  337077
incoming                   99695
forward                     8573
originate                   5483
voicemailexit               3929
queue-exit                   154
cisco-3way                    50
default                       30
conferencing-originate        23
v5-to-v4                       1
confirm-callee                 1
Name: type, dtype: int64

In [157]:
dfSep['month'].value_counts()

9    240828
Name: month, dtype: int64

In [201]:
def calculateAndMerge():
    dfConcat = pd.concat([dfJul, dfAgo, dfSep])

    dfConcat.describe()

    dfGroup = dfConcat.groupby(["pbx_id","month","type"])

    gMean = dfGroup.mean(numeric_only=True).add_suffix("_Mean").reset_index()

    gSum = dfGroup.sum(numeric_only=True).add_suffix("_Sum").reset_index()

    gMax = dfGroup.max(numeric_only=True).add_suffix("_Max").reset_index()

    gMin = dfGroup.min(numeric_only=True).add_suffix("_Min").reset_index()
    
    mergedDf = pd.merge(gSum, gMean ,how='inner',left_on=['pbx_id','month','type'], right_on=['pbx_id', 'month','type'])

    mergedDf = pd.merge(mergedDf, gMax ,how='inner',left_on=['pbx_id','month','type'], right_on=['pbx_id', 'month','type'])

    mergedDf = pd.merge(mergedDf, gMin ,how='inner',left_on=['pbx_id','month','type'], right_on=['pbx_id', 'month','type'])
    
    mergedDf.drop(['year_Sum', "day_Sum", "year_Mean", "day_Mean", "year_Max", "day_Max", "year_Min", "day_Min"], axis=1, inplace=True)
    
    return mergedDf.copy()
    

In [202]:
myData = calculateAndMerge()


In [208]:
myData.to_csv('usage.csv', index=False)

In [209]:
myData = pd.read_csv('usage.csv')

In [233]:
s =myData [ myData['pbx_id'] == "0022298d-7115-4315-a863-d3ae0fca517e" ]

In [234]:
s


,pbx_id,month,type,calls_Sum,minutes_Sum,calls_Mean,minutes_Mean,calls_Max,minutes_Max,calls_Min,minutes_Min
3,0022298d-7115-4315-a863-d3ae0fca517e,8,outgoing,139,1041,6.950000,52.050000,14,134,1,0
4,0022298d-7115-4315-a863-d3ae0fca517e,9,incoming,3,5,1.500000,2.500000,2,4,1,1
5,0022298d-7115-4315-a863-d3ae0fca517e,9,outgoing,49,347,5.444444,38.555556,10,114,1,4


In [232]:
b =s[['type','month']]


In [241]:
s[s['month'].apply(lambda x: isInArray(x))]

,pbx_id,month,type,calls_Sum,minutes_Sum,calls_Mean,minutes_Mean,calls_Max,minutes_Max,calls_Min,minutes_Min
4,0022298d-7115-4315-a863-d3ae0fca517e,9,incoming,3,5,1.500000,2.500000,2,4,1,1
5,0022298d-7115-4315-a863-d3ae0fca517e,9,outgoing,49,347,5.444444,38.555556,10,114,1,4


In [239]:
def isInArray(x):
    return x in [1,2,9]

In [244]:
x = ['pbx_id', 'month', 'type', 'calls_Sum', 'minutes_Sum']
s[x]

,pbx_id,month,type,calls_Sum,minutes_Sum
3,0022298d-7115-4315-a863-d3ae0fca517e,8,outgoing,139,1041
4,0022298d-7115-4315-a863-d3ae0fca517e,9,incoming,3,5
5,0022298d-7115-4315-a863-d3ae0fca517e,9,outgoing,49,347
